# How to classify new sequences?

This notebook contains a tutorial that shows you how to classify new sequences.

### Input data
* Camera trap images, in folders per deployment
* Agouti export files: observations, assets and pickup-setup

### Load the predefined configuration file

This file contains the paths to the different folders. <br>
If you have the same data structure as in this repository, you can use the predefined configuration file. If needed, you can change the paths according to you own folder structure.

The following paths are defined:
* **general_folder_path** : orginal camera trap images and the Agouti export files (assets, observations and pickup-setup)
* **resized_folder_path** : resized camera trap images
* **preprocessing_output_path** : preprocessing output
* **crop_output_path** : cropped images (optional)
* **draw_output_path** : regions of interest indicated on original camera trap images (optional)
* **bottleneck_features_output_path** : extracted bottleneck features
* **weight_path** : weights top model
* **predictions_output_path** : predictions

In [1]:
from yaml import load

with open("config.yml") as yaml_config:
    config = load(yaml_config)

c:\users\emma_cartuyvels\appdata\local\continuum\anaconda3\envs\cameratraps\lib\site-packages\ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


### Check if all folders exist and create folder if needed

In [2]:
import os

for path in config:
    if not os.path.exists(config[path]):
        os.makedirs(config[path])

### Step 1: resize images
The camera trap images are resized to 50% of their original size. This strongly decreases the computational time, while the performance remains the same.<br>

Input: original camera trap images and Agouti export file (observations)<br>
Output: resized camera trap images in a similar folder structure as the original images

In [3]:
from preprocessing.resize_images import resize_images

resize_images(config["general_folder_path"], 
              config["resized_folder_path"])

### Step 2: preprocess images
During the preprocessing, the regions of interest in the images are determined. All images of a sequence are used to construct a background image. Subsequently, the regions of interest in a camera trap image are determined by computing the difference between this background image and the camera trap image.

Input: resized camera trap images and Agouti export files(observations + assets + pickupsetup)<br>
Output: cvs-file containing the coordinates of the regions of interest in every camera trap image

In [14]:
import os
import numpy as np
import pandas as pd

from PIL import Image, ImageChops, ImageFilter
from skimage import measure
from skimage.filters import roberts
from scipy import ndimage as ndi
import cv2

from preprocessing.def_functions import remove_dup_columns, black_border, standard_box, size_box, devide_box

general_folder_path = '../data/raw'

#Resized images
RATIO = 0.5
    
#Import Agouti export files
observations = pd.read_csv(os.path.join(general_folder_path, 'observations.csv'))
assets = pd.read_csv(os.path.join(general_folder_path, 'assets.csv'), low_memory = False)
setup = pd.read_csv(os.path.join(general_folder_path, 'pickup_setup.csv'))
    
#Combine annotations for sequences with multiple annotations
list_columns = ['animalCount','animalTaxonID','animalIsDomesticated','animalScientificName','animalVernacularName','animalSex','animalAge', 'animalBehavior', 'deploymentID']
observations_unique = pd.DataFrame()   
for name in list_columns:
    column_unique = observations.groupby('sequenceID')[name].apply(list).reset_index()
    observations_unique = pd.concat([observations_unique, column_unique], axis=1)
observations_unique = remove_dup_columns(observations_unique)
    
#Join annotations and pickup-setup data
ann = assets.set_index('sequence').join(observations_unique.set_index('sequenceID'))
ann.index.name = 'sequenceId'
data = ann.merge(setup, on='sequenceId', how='left')
data

,sequenceId,Unnamed: 0_x,id,type,filename,originalFilename,destination,directory,exiftoolData,order,...,animalVernacularName,animalSex,animalAge,animalBehavior,deploymentID,Unnamed: 0_y,deploymentId,isBlank,isSetupPickup,isTimeLapse
0,000fb530-eb06-4017-abc5-cb5bd6ad3c45,71705,NaN,image,NaN,RCNX7763.JPG,NaN,NaN,NaN,NaN,...,[Eurasian Coot],[nan],[nan],[{}],[2adb607a-7e6a-4919-8702-fdf514bd3bd5],3059.0,2adb607a-7e6a-4919-8702-fdf514bd3bd5,NaN,NaN,ONWAAR
1,000fb530-eb06-4017-abc5-cb5bd6ad3c45,71706,NaN,image,NaN,RCNX7764.JPG,NaN,NaN,NaN,NaN,...,[Eurasian Coot],[nan],[nan],[{}],[2adb607a-7e6a-4919-8702-fdf514bd3bd5],3059.0,2adb607a-7e6a-4919-8702-fdf514bd3bd5,NaN,NaN,ONWAAR
2,000fb530-eb06-4017-abc5-cb5bd6ad3c45,71707,NaN,image,NaN,RCNX7765.JPG,NaN,NaN,NaN,NaN,...,[Eurasian Coot],[nan],[nan],[{}],[2adb607a-7e6a-4919-8702-fdf514bd3bd5],3059.0,2adb607a-7e6a-4919-8702-fdf514bd3bd5,NaN,NaN,ONWAAR
3,000fb530-eb06-4017-abc5-cb5bd6ad3c45,71708,NaN,image,NaN,RCNX7766.JPG,NaN,NaN,NaN,NaN,...,[Eurasian Coot],[nan],[nan],[{}],[2adb607a-7e6a-4919-8702-fdf514bd3bd5],3059.0,2adb607a-7e6a-4919-8702-fdf514bd3bd5,NaN,NaN,ONWAAR
4,000fb530-eb06-4017-abc5-cb5bd6ad3c45,71709,NaN,image,NaN,RCNX7767.JPG,NaN,NaN,NaN,NaN,...,[Eurasian Coot],[nan],[nan],[{}],[2adb607a-7e6a-4919-8702-fdf514bd3bd5],3059.0,2adb607a-7e6a-4919-8702-fdf514bd3bd5,NaN,NaN,ONWAAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364587,NaN,344437,NaN,image,NaN,RCNX1005.JPG,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364588,NaN,344438,NaN,image,NaN,RCNX1010.JPG,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364589,NaN,344439,NaN,image,NaN,RCNX1014.JPG,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364590,NaN,344440,NaN,image,NaN,RCNX1013.JPG,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
from preprocessing.preprocessing import preprocessing

preprocessing(config["general_folder_path"], 
              config["resized_folder_path"], 
              config["preprocessing_output_path"])

KeyError: 'sequenceId'

#### Optional: crop images or indicate regions of interest
We can crop the camera trap images or indicate the regions of interest on the camera trap images to see the result of the preprocessing. <br>
This step is optional and not required to classify the images.

In [ ]:
from preprocessing.crop_images import crop_images
from preprocessing.draw_boxes import draw_boxes

crop_images(config["preprocessing_output_path"], config["resized_folder_path"], config["crop_output_path"])
draw_boxes(config["preprocessing_output_path"], config["resized_folder_path"], config["draw_output_path"])

In [ ]:
from PIL import image

# Show a camera trap image with indication of the regions of interest
deployment = os.listdir(config["draw_output_path"])[0]
image_name = os.listdir(deployment)[0]
image = Image.open(os.path.join(config["draw_output_path"],deployment, image_name))
image

### Step 3: extract bottleneck features

The pretrained convolutional neural network ResNet50 is used to convert the images to bottleneck features.

Input: resized camera trap images and preprocessing output containing the coordinates of the boxes <br>
Output: bottleneck features

In [ ]:
from network.resnet50_bottleneck_features_predict import extract_bottleneck_features

extract_bottleneck_features(config["preprocessing_output_path"], 
                            config["bottleneck_features_output_path"], 
                            config["resized_folder_path"])

### Step 4 : run top model to classify the new images

The extracted bottleneck features are fed to the new top model to predict the labels of the new images.

Input: extracted bottleneck features <br>
Ouput: probabilities of the output classes

In [ ]:
from network.resnet50_hierarchical_bottleneck_predict import hierarchical_bottleneck_predict

hierarchical_bottleneck_predict(config["bottleneck_features_output_path"], 
                                config["weight_path"], 
                                config["predictions_output_path"])

### Step 5 : convert output probabilities to hierarchical classification

The predictions of the individual images are aggregated to a hierarchical prediction for every sequence.

Input: probabilities of the output classes for the individual images <br>
Output: hierarchical classification of the sequences

In [ ]:
from network.hierachical_processing_predictions import hierarchical_predictions_sequences

hierarchical_predictions_sequences( config["predictions_output_path"], 
                                   config["bottleneck_features_output_path"])

We can load the file containing the hierarchical predictions to see the predicted labels.

In [ ]:
import pandas as pd

predictions = pd.read_csv(os.path.join(config["predictions_output_path"],'hierarchical_predictions_sequences.csv'), sep = ';')
predictions.drop(['level_1_p','level_2_p','level_3_p','level_4_p','level_5_p'], axis=1, inplace=True)

print(predictions)

### Optional: Object localization

The class activation maps can be used to localize the objects in the cropped camera trap images. <br>
Since this step uses the cropped images, make sure to first run the optional cropping step above.

In [ ]:
from network.resnet_cam import object_localization

img_path = os.path.join(config["crop_output_path"], os.listdir(config["crop_output_path"])[0])
object_localization(img_path)